In [ ]:
#what i need to do is to 

# 1. make gmaps query “company name san francisco” --> 
# 2. let a agent run over this and have him extract all the addresses where he actually thinks this could be the address of an actual office
# 3. create the interactive map for san francisco where each pin represents a company - the pin is the company profile pic and on click of it all the other metadata is displayed

In [16]:
#load all the offices which are located in san francisco and which are still active today
import json

def load_and_filter_companies(file_path):
    active_sf_companies = []

    with open(file_path, 'r') as json_file:
        for line in json_file:
            company = json.loads(line)
            if company.get('status') == "Active" and company.get('location') == "San Francisco":
                active_sf_companies.append(company)

    return active_sf_companies

# Path to your dataset
file_path = 'company_data.json'

# Load and filter the dataset
filtered_companies = load_and_filter_companies(file_path)

# Print or save the filtered results
for company in filtered_companies:
    print(json.dumps(company, indent=4))

# Optionally, save the filtered results to a new JSON file
with open('active_sf_companies.json', 'w') as json_file:
    for company in filtered_companies:
        json_file.write(json.dumps(company) + '\n')


{
    "url": "https://www.ycombinator.com/companies/jet-admin",
    "name": "Jet Admin",
    "status": "Active",
    "profile_picture_url": "https://bookface-images.s3.amazonaws.com/small_logos/36f3d929390edbb2302940824ff50512f48f5516.png",
    "tags": [
        "Y Combinator LogoW20",
        "developer-tools",
        "saas",
        "b2b",
        "San Francisco"
    ],
    "description": "Jet Admin - an app builder designed specifically for creating internal tools, admin panels, external portals, inventory apps, and other business applications really fast and without coding. \r\n\r\nIn Jet Admin, you can assemble multi-page apps with complex user flows, connected to multiple data sources. Setting up custom layouts, adding custom actions and workflows, binding and transforming dynamic values, using conditional rules and validations - all is done through a drag & drop and point & click visual builder.\r\n\r\nWe\u2019ve also added a variety of developer tools, so whenever necessary, y

In [32]:
#create all queries for the google maps text search
import json
import os
import asyncio
import aiohttp
from dotenv import load_dotenv
import nest_asyncio

# Apply the nest_asyncio patch
nest_asyncio.apply()

load_dotenv()

# Retrieve API key from environment variables
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

BASE_URL = "https://places.googleapis.com/v1/places:searchText"

async def search_places(session, query, retries=3):
    print("query:", query)
    print("session:", session)

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': API_KEY,
        'X-Goog-FieldMask': 'places.displayName,places.formattedAddress,places.types,places.websiteUri,places.rating,places.userRatingCount'
    }
    body = {
        "textQuery": query,
        "pageSize": 20  # Set to maximum number of results
    }
    for attempt in range(retries):
        try:
            async with session.post(BASE_URL, headers=headers, json=body) as response:
                if response.status == 200:
                    data = await response.json()
                    if 'places' in data:
                        return {"query": query, "results": data['places']}
                    else:
                        print(f"No results found in response for query: {query}")
                        print("Response data:", data)
                        return {"query": query, "results": []}
                elif response.status == 429:  # Too Many Requests
                    print(f"Error: {response.status}. Retrying in {2 ** attempt} seconds...")
                    await asyncio.sleep(2 ** attempt)
                else:
                    response.raise_for_status()
        except aiohttp.ClientError as e:
            print(f"Request failed: {e}. Retrying in {2 ** attempt} seconds...")
            await asyncio.sleep(2 ** attempt)
    return {"query": query, "results": []}

# Load the filtered companies from the JSON file
def load_filtered_companies(file_path):
    with open(file_path, 'r') as json_file:
        return [json.loads(line) for line in json_file]

# Create a list of queries with the company name + "San Francisco"
def create_queries(companies):
    return [f"{company['name']} San Francisco" for company in companies]

# Path to the filtered dataset
file_path = 'active_sf_companies.json'

# Load the filtered companies
filtered_companies = load_filtered_companies(file_path)

# Create the list of queries
queries = create_queries(filtered_companies)    

In [39]:
import aiohttp
import asyncio

async def search_places(query, retries=3):
    url = BASE_URL
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': API_KEY,
        'X-Goog-FieldMask': 'places.displayName,places.formattedAddress,places.types,places.websiteUri,places.rating,places.userRatingCount'
    }
    body = {
        "textQuery": query,
        "pageSize": 20  # Set to maximum number of results
    }
    for attempt in range(retries):
        try:
            async with aiohttp.ClientSession() as session:
                async with session.post(url, headers=headers, json=body) as response:
                    if response.status == 200:
                        data = await response.json()
                        if 'places' in data:
                            # Print the response to verify the structure
                            #print(f"Response for query '{query}':", data['places'])
                            return {"query": query, "results": data['places']}
                        else:
                            #print(f"No results found in response for query: {query}")
                            #print("Response data:", data)
                            return {"query": query, "results": []}
                    elif response.status == 429:  # Too Many Requests
                        print(f"Error: {response.status}. Retrying in {2 ** attempt} seconds...")
                        await asyncio.sleep(2 ** attempt)
                    else:
                        response.raise_for_status()
        except aiohttp.ClientError as e:
            print(f"Request failed: {e}. Retrying in {2 ** attempt} seconds...")
            await asyncio.sleep(2 ** attempt)
    return {"query": query, "results": []}

# Example usage
for query in queries:
    response = await search_places(query)
    print("repsonse:", response)
    await asyncio.sleep(0.5)  # Wait for 500 milliseconds
    with open('search_results.json', 'a') as json_file:
        json.dump(response, json_file, indent=4)


repsonse: {'query': 'Jet Admin San Francisco', 'results': [{'types': ['point_of_interest', 'establishment'], 'formattedAddress': 'Alemany Blvd, San Francisco, CA 94112, USA', 'websiteUri': 'https://www.jetadmin.io/', 'displayName': {'text': 'JetAdmin', 'languageCode': 'en'}}]}
repsonse: {'query': 'Legionfarm San Francisco', 'results': [{'types': ['point_of_interest', 'establishment'], 'formattedAddress': '401 Van Ness Ave #101, San Francisco, CA 94102, USA', 'rating': 3, 'websiteUri': 'https://alwmcsf.org/', 'userRatingCount': 1, 'displayName': {'text': 'American Legion War Memorial Commission', 'languageCode': 'en'}}]}
repsonse: {'query': 'Menten AI San Francisco', 'results': [{'types': ['point_of_interest', 'establishment'], 'formattedAddress': '3154 17th St, San Francisco, CA 94110, USA', 'rating': 5, 'websiteUri': 'https://www.mindsdb.com/', 'userRatingCount': 2, 'displayName': {'text': 'MindsDB SF AI Collective', 'languageCode': 'en'}}]}
repsonse: {'query': 'Equator Therapeutics S

In [ ]:
#now w